# Computational Theory Assessment — SHA-256 (FIPS 180-4)

**Student:** Adam Gallagher 

**Student ID:** G00413950

This notebook implements core components of SHA-256 directly from the Secure Hash Standard (FIPS 180-4), using NumPy `uint32` to preserve 32-bit word semantics (wraparound arithmetic and bitwise behavior).


In [34]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


## Scope and structure

This notebook is organised into five problems aligned to the Secure Hash Standard:

1. Implement SHA-256 word/bitwise primitives (`Ch`, `Maj`, Σ/σ functions, etc.)
2. Derive SHA-256 round constants from fractional cube roots of primes
3. Implement SHA-256 padding and 512-bit block parsing (generator)
4. Implement SHA-256 compression step `hash(current, block)` (Section 6.2.2)
5. Demonstrate a dictionary attack on unsalted one-pass SHA-256 password hashes and discuss mitigations


## Validation strategy

I validate correctness at multiple levels:

- Primitive functions are tested against their defining bitwise identities.
- Computed SHA-256 constants `K[0..63]` are compared to the constants listed in FIPS 180-4 (page 11).
- Padding/block parsing is tested on boundary-length inputs (0, 1, 55, 56, 63, 64 bytes) and verified to encode the correct original bit length.
- An end-to-end SHA-256 implementation (Problems 1–4) is compared to Python’s `hashlib.sha256` on multiple inputs.


In [35]:
# Imports
import numpy as np
import math
import hashlib

np.seterr(over="ignore")
np.set_printoptions(formatter={"int": lambda x: f"{x:#010x}"})



# Problem 1: Binary Words and Operations

**What the problem is**  
- Implement SHA-256 boolean functions and Σ/σ functions using NumPy `uint32` so all operations behave as 32-bit unsigned arithmetic. Document and test each function.

**How I’m going to solve it**  
- Use `np.uint32` for all inputs/outputs.  
- Implement rotate-right (`ROTR`) and logical right shift (`SHR`).  
- Implement `Parity`, `Ch`, `Maj`, `Σ0`, `Σ1`, `σ0`, `σ1` exactly as defined in FIPS 180-4.  
- Add docstrings and minimal correctness tests.


In [36]:
# ---- 32-bit utilities ----

def u32(x) -> np.uint32:
    """Cast to 32-bit unsigned integer."""
    return np.uint32(x)

def rotr(x: np.uint32, n: int) -> np.uint32:
    """
    Rotate-right for 32-bit words.
    ROTR^n(x) = (x >> n) OR (x << (32-n)) with 32-bit wraparound.
    """
    x = u32(x)
    return u32((x >> n) | (x << (32 - n)))

def shr(x: np.uint32, n: int) -> np.uint32:
    """
    Logical right shift for 32-bit words.
    For uint32, right shift is logical (zeros shifted in).
    """
    return u32(u32(x) >> n)

# ---- Problem 1 required functions (FIPS 180-4) ----

def Parity(x: np.uint32, y: np.uint32, z: np.uint32) -> np.uint32:
    """
    Parity(x,y,z) = x XOR y XOR z.
    """
    return u32(u32(x) ^ u32(y) ^ u32(z))

def Ch(x: np.uint32, y: np.uint32, z: np.uint32) -> np.uint32:
    """
    Choice function:
    Ch(x,y,z) = (x AND y) XOR ((NOT x) AND z).
    Bitwise: chooses y when x=1, else chooses z.
    """
    return u32((u32(x) & u32(y)) ^ (~u32(x) & u32(z)))

def Maj(x: np.uint32, y: np.uint32, z: np.uint32) -> np.uint32:
    """
    Majority function:
    Maj(x,y,z) = (x AND y) XOR (x AND z) XOR (y AND z).
    Bitwise majority vote across x,y,z.
    """
    return u32((u32(x) & u32(y)) ^ (u32(x) & u32(z)) ^ (u32(y) & u32(z)))

def Sigma0(x: np.uint32) -> np.uint32:
    """
    Σ0(x) = ROTR^2(x) XOR ROTR^13(x) XOR ROTR^22(x).
    """
    return u32(rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22))

def Sigma1(x: np.uint32) -> np.uint32:
    """
    Σ1(x) = ROTR^6(x) XOR ROTR^11(x) XOR ROTR^25(x).
    """
    return u32(rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))

def sigma0(x: np.uint32) -> np.uint32:
    """
    σ0(x) = ROTR^7(x) XOR ROTR^18(x) XOR SHR^3(x).
    """
    return u32(rotr(x, 7) ^ rotr(x, 18) ^ shr(x, 3))

def sigma1(x: np.uint32) -> np.uint32:
    """
    σ1(x) = ROTR^17(x) XOR ROTR^19(x) XOR SHR^10(x).
    """
    return u32(rotr(x, 17) ^ rotr(x, 19) ^ shr(x, 10))

In [37]:
# Problem 1: minimal correctness tests (bitwise identities and rotation/shift sanity)

x, y, z = u32(0x0F0F0F0F), u32(0x33333333), u32(0xAAAAAAAA)

assert Parity(x, y, z) == u32(x ^ y ^ z)
assert Ch(x, y, z) == u32((x & y) ^ (~x & z))
assert Maj(x, y, z) == u32((x & y) ^ (x & z) ^ (y & z))

# Rotation/shift sanity checks
assert rotr(u32(0x80000000), 1) == u32(0x40000000)
assert rotr(u32(0x00000001), 1) == u32(0x80000000)
assert shr(u32(0x80000000), 1) == u32(0x40000000)

print("Problem 1 tests passed.")

Problem 1 tests passed.


# Problem 2: Fractional Parts of Cube Roots

**What the problem is**  
Recompute the SHA-256 constants listed on page 11 of FIPS 180-4: the first 32 bits of the fractional parts of the cube roots of the first 64 primes.

**How I’m going to solve it**  
- Write `primes(n)` to generate the first `n` primes.  
- Compute cube roots of the first 64 primes.  
- Extract fractional part, multiply by `2^32`, take floor, cast to `np.uint32`.  
- Display results in hex and compare against the standard.


In [38]:
def primes(n: int) -> list[int]:
    """
    Generate the first n prime numbers using trial division.
    """
    if n < 0:
        raise ValueError("n must be non-negative")
    out = []
    x = 2
    while len(out) < n:
        if x == 2:
            out.append(x)
        elif x % 2 == 0:
            pass
        else:
            is_p = True
            r = int(math.isqrt(x))
            for d in range(3, r + 1, 2):
                if x % d == 0:
                    is_p = False
                    break
            if is_p:
                out.append(x)
        x += 1
    return out

def sha256_K_from_cuberoot_primes() -> np.ndarray:
    """
    Compute SHA-256 K constants:
    K[i] = floor( frac(cuberoot(p_i)) * 2^32 ) as uint32,
    where p_i is the i-th prime, i=0..63.
    """
    ps = primes(64)
    K_calc = np.zeros(64, dtype=np.uint32)
    for i, p in enumerate(ps):
        root = p ** (1.0 / 3.0)
        frac = root - math.floor(root)
        K_calc[i] = np.uint32(math.floor(frac * (2**32)))
    return K_calc

K_calc = sha256_K_from_cuberoot_primes()
hex_list = [f"{int(v):08x}" for v in K_calc]
hex_list[:8], hex_list[-8:]


(['428a2f98',
  '71374491',
  'b5c0fbcf',
  'e9b5dba5',
  '3956c25b',
  '59f111f1',
  '923f82a4',
  'ab1c5ed5'],
 ['748f82ee',
  '78a5636f',
  '84c87814',
  '8cc70208',
  '90befffa',
  'a4506ceb',
  'bef9a3f7',
  'c67178f2'])

In [39]:
# Standard SHA-256 K constants (as listed in FIPS 180-4 page 11)
K = np.array([
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
], dtype=np.uint32)

assert np.all(K_calc == K), "Computed K constants do not match the standard."
print("Problem 2 tests passed: computed K matches the standard.")


Problem 2 tests passed: computed K matches the standard.


# Problem 3: Padding

**What the problem is**  
Write a generator `block_parse(msg)` that yields 512-bit blocks (64 bytes) of the message after applying SHA-256 padding (Sections 5.1.1 and 5.2.1).

**How I’m going to solve it**  
- Compute original message bit-length.  
- Append `0x80`, then `0x00` until the length is 56 bytes mod 64.  
- Append the 64-bit big-endian length.  
- Yield successive 64-byte blocks.

In [40]:
def block_parse(msg: bytes):
    """
    Yield 512-bit blocks (64 bytes) from msg after applying SHA-256 padding.

    Padding:
      - append 1 bit (0x80)
      - append zeros until length ≡ 56 (mod 64)
      - append 64-bit big-endian original bit length
    """
    if not isinstance(msg, (bytes, bytearray)):
        raise TypeError("msg must be bytes-like")

    bit_len = len(msg) * 8

    padded = bytearray(msg)
    padded.append(0x80)

    while (len(padded) % 64) != 56:
        padded.append(0x00)

    padded.extend(bit_len.to_bytes(8, byteorder="big", signed=False))

    for i in range(0, len(padded), 64):
        yield bytes(padded[i:i+64])

In [41]:
# Problem 3 tests: boundary lengths and length field validation

def pad_bytes(msg: bytes) -> bytes:
    return b"".join(block_parse(msg))

# Boundary block-count tests
assert len(list(block_parse(b""))) == 1
assert len(list(block_parse(b"a"*55))) == 1
assert len(list(block_parse(b"a"*56))) == 2
assert len(list(block_parse(b"a"*63))) == 2
assert len(list(block_parse(b"a"*64))) == 2

# Check all yielded blocks are 64 bytes
for m in [b"", b"a", b"a"*55, b"a"*56, b"a"*63, b"a"*64]:
    blocks = list(block_parse(m))
    assert all(len(b) == 64 for b in blocks)

# Length field correctness for a sample
m = b"abc"
p = pad_bytes(m)
assert int.from_bytes(p[-8:], "big") == len(m) * 8

print("Problem 3 tests passed.")


Problem 3 tests passed.


# Problem 4: Hashes

**What the problem is**  
Implement the SHA-256 compression step `hash(current, block)` that computes the next hash state from the current 8-word state and the next 512-bit message block (Section 6.2.2, page 22).

**How I’m going to solve it**  
- Parse block into 16 big-endian 32-bit words and build schedule `W[0..63]`.  
- Initialize working variables `a..h` from `current`.  
- Run 64 rounds using `Ch`, `Maj`, `Σ0`, `Σ1`, constants `K[t]`, and the schedule.  
- Add working variables back into `current` and return the next state.  
- Validate end-to-end against `hashlib.sha256`.

In [42]:
def hash(current: np.ndarray, block: bytes) -> np.ndarray:
    """
    SHA-256 compression function: compute the next hash state.

    Parameters
    ----------
    current : np.ndarray shape (8,), dtype uint32
        Current intermediate hash value H0..H7.
    block : bytes length 64
        Next 512-bit message block.

    Returns
    -------
    np.ndarray shape (8,), dtype uint32
        Next intermediate hash value.
    """
    if len(block) != 64:
        raise ValueError("block must be exactly 64 bytes.")
    current = np.array(current, dtype=np.uint32)
    if current.shape != (8,):
        raise ValueError("current must have shape (8,)")

    # Message schedule W[0..63]
    M = np.frombuffer(block, dtype=">u4").astype(np.uint32)  # 16 big-endian words
    W = np.zeros(64, dtype=np.uint32)
    W[:16] = M
    for t in range(16, 64):
        W[t] = u32(sigma1(W[t-2]) + W[t-7] + sigma0(W[t-15]) + W[t-16])

    a, b, c, d, e, f, g, h = current

    for t in range(64):
        T1 = u32(h + Sigma1(e) + Ch(e, f, g) + K[t] + W[t])
        T2 = u32(Sigma0(a) + Maj(a, b, c))
        h = g
        g = f
        f = e
        e = u32(d + T1)
        d = c
        c = b
        b = a
        a = u32(T1 + T2)

    return np.array([
        u32(current[0] + a),
        u32(current[1] + b),
        u32(current[2] + c),
        u32(current[3] + d),
        u32(current[4] + e),
        u32(current[5] + f),
        u32(current[6] + g),
        u32(current[7] + h),
    ], dtype=np.uint32)


In [43]:
# End-to-end SHA-256 using Problems 1–4

H0 = np.array([
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
], dtype=np.uint32)

def sha256_digest(msg: bytes) -> bytes:
    """
    Compute SHA-256 digest bytes using this notebook's implementation.
    """
    state = H0.copy()
    for blk in block_parse(msg):
        state = hash(state, blk)
    return b"".join(int(w).to_bytes(4, "big") for w in state.tolist())

def sha256_hexdigest(msg: bytes) -> str:
    return sha256_digest(msg).hex()

# Known-answer tests (compare against hashlib)
for m in [b"", b"a", b"abc", b"hello world", b"a"*55, b"a"*56, b"a"*64]:
    mine = sha256_hexdigest(m)
    ref = hashlib.sha256(m).hexdigest()
    assert mine == ref, f"Mismatch for message {m!r}"

print("Problem 4 tests passed: implementation matches hashlib on test inputs.")


Problem 4 tests passed: implementation matches hashlib on test inputs.


# Problem 5: Passwords

**What the problem is**  
Given three SHA-256 digests of UTF-8 encoded passwords (one SHA-256 pass), determine the passwords, explain the method, and propose improvements to prevent the attack.

**How I’m going to solve it**  
- Perform an offline dictionary attack: hash candidate passwords with SHA-256 (UTF-8) and compare digests.  
- Verify recovered passwords by recomputing hashes.  
- Discuss mitigations: unique per-user salt and a slow password-hashing/KDF approach (plus optional pepper and operational controls).


In [44]:
# The following are the SHA-256 hashes of three common passwords that have been hashed using one pass of the SHA-256 algorithm. As strings, they were encoded using UTF-8. 
# Determine the passwords and explain how you found them. Suggest ways in which the hashing of passwords could be improved to prevent the kind of attack you performed to find the passwords.

    # 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
    # 873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34
    # b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342